In [210]:
import Pkg
Pkg.add("POMDPs")
Pkg.add("POMDPLinter")
Pkg.add("POMDPModels")
Pkg.add("POMDPTools")
Pkg.add("POMDPPolicies")
Pkg.add("IterTools")
Pkg.add("DiscreteValueIteration")

   Resolving package versions...
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\surbh\.julia\environments\v1.10\Project.toml`
  No 

In [211]:
using POMDPs
using IterTools

In [212]:

# Define a struct for an individual object's state using NamedTuples
struct ObjectState
    attributes::NamedTuple # Attributes of each object stored as NamedTuple
end

# Define a struct for the factored state space using NamedTuples
struct MyFactoredState
    attributes::NamedTuple # State-wide attributes stored as NamedTuple
    objects::NamedTuple    # NamedTuple to store multiple objects dynamically
end


In [213]:

# Define the state space function
function POMDPs.states(::Type{<:MyFactoredState}, object_ranges::NamedTuple, state_attr_ranges::NamedTuple)
    # Generate all possible combinations for state attributes using IterTools
    attribute_combinations = Iterators.product(state_attr_ranges...)

    # List to store all possible states
    # state_space = []
    state_space = Vector{MyFactoredState}()

    # Iterate over each combination of state attributes
    for attr_comb in attribute_combinations
        # Create a NamedTuple for the state-wide attributes
        state_attrs = NamedTuple(zip(keys(state_attr_ranges), attr_comb))

        # Create all combinations for the objects' attributes
        object_combinations = generate_object_combinations(object_ranges)

        # Iterate over the object configurations
        for obj_comb in object_combinations
            # Create a NamedTuple for each object
            objects_namedtuple = NamedTuple((obj_name => ObjectState(NamedTuple(zip(keys(object_attr), obj)))) 
                                             for (obj_name, object_attr, obj) in zip(keys(object_ranges), values(object_ranges), obj_comb))
            
            # Push the new state to the state space
            push!(state_space, MyFactoredState(state_attrs, objects_namedtuple))
        end
    end
    # push error state 
    push!(state_space,  MyFactoredState(
        NamedTuple((Redball1Matched = -1, Blackball1Matched = -1)),
        NamedTuple((Redball1 = ObjectState((pos = -1,)), Blackball1 = ObjectState((pos = -1,)), Redbox1 = ObjectState((pos = -1, )), Blackbox1 = ObjectState((pos = -1,))))
    ))
    return state_space
end

# Helper function to generate all possible combinations for objects' attributes
function generate_object_combinations(object_ranges::NamedTuple)
    # List to store all combinations for the objects
    object_combinations = []

    # Create iterators for all object attribute combinations
    for object_attr_range in values(object_ranges)
        object_combinations_for_one = Iterators.product(object_attr_range...)
        push!(object_combinations, object_combinations_for_one)
    end

    # Combine all object combinations into a single iterator
    combined_combinations = Iterators.product(object_combinations...)
    
    return combined_combinations
end



generate_object_combinations (generic function with 1 method)

In [214]:


# Define the object attributes and ranges
object_ranges = NamedTuple((
    Redball1 = NamedTuple((pos = 1:6 , isMovable = 0:1)),  # Object o1 attributes
    Redball2 = NamedTuple((pos = 1:6 , isMovable = 0:1)),  # Object o2 attributes 
    Blackball1 = NamedTuple((pos = 1:6 , isMovable = 0:1)), 
    Blackball2 = NamedTuple((pos = 1:6 , isMovable = 0:1)),
    Redbox1 = NamedTuple((pos = 1:6 , isMovable = 0:1)),
    Blackbox1 = NamedTuple((pos = 1:6 , isMovable = 0:1))
    ))

# object_ranges = NamedTuple((
#     Redball1 = NamedTuple((pos = 1:4, )),  # Object o1 attributes 
#     Blackball1 = NamedTuple((pos = 1:4 ,)),
#     Redbox1 = NamedTuple((pos = 1:4, )),
#     Blackbox1 = NamedTuple((pos = 1:4, ))
# ))

state_attr_ranges = NamedTuple((
    Redball1Matched = 0:1,
    Blackball1Matched = 0:1 
))



# Define the global state attributes and ranges
# state_attr_ranges = NamedTuple((
#     Redball1Matched = 0:1,
#     Redball2Matched = 0:1,
#     Blackball1Matched = 0:1,
#     Blackball2Matched = 0:1,
#     RedballsMatched = 0:2,
#     BlackballsMatched = 0:2 
# ))

# Generate the state space
# state_space = POMDPs.states(MyFactoredState, object_ranges, state_attr_ranges)


(Redball1Matched = 0:1, Blackball1Matched = 0:1)

In [215]:
function temp_isterminal(s::MyFactoredState) 
    return s.attributes.Redball1Matched == 1 && s.attributes.Blackball1Matched == 1 
end

# Function to find all goal states in the MDP
function find_goal_states(state_array::Vector{MyFactoredState})
    # Access the precomputed state array
 
    # Initialize an empty vector to store goal states
    goal_states = Vector{MyFactoredState}()

    # Iterate over each state in the state array
    for state in state_array
        # Check if the state is a terminal (goal) state
        if temp_isterminal(state)
            push!(goal_states, state) # Add to goal states vector
        end
    end

    return goal_states
end


find_goal_states (generic function with 1 method)

In [216]:

# Define an Action struct with two attributes of type Symbol
struct MatchAction
    attr1::Symbol  # obj1 
    attr2::Symbol  # obj2 
end

# function temp_actions() 
#     return [MatchAction(:Redball1, :Redbox1), MatchAction(:Blackball1, :Redbox1), MatchAction(:Redball2, :Redbox1), MatchAction(:Blackball2, :Redbox1), MatchAction(:Redball1, :Blackbox1), MatchAction(:Blackball1, :Blackbox1), MatchAction(:Redball2, :Blackbox1), MatchAction(:Blackball2, :Blackbox1)] 
# end

function temp_actions()
    return [MatchAction(:Redball1, :Redbox1), MatchAction(:Blackball1, :Redbox1), MatchAction(:Redball1, :Blackbox1), MatchAction(:Blackball1, :Blackbox1)]
end



temp_actions (generic function with 1 method)

In [234]:


struct GameMDP <: MDP{MyFactoredState, MatchAction}
    # reward_states_values::Dict{MyFactoredState, Float64} # Dictionary mapping reward states to their values
    goal_states::Vector{MyFactoredState} 
    # tprob::Float64 # probability of transitioning to the desired state
    discount_factor::Float64 # discount factor
    state_array::Vector{MyFactoredState} # vector to store all states in the MDP
    actions::Vector{MatchAction} # vector to store all actions in the MDP 
end 


function GameMDP()
    # Precompute the state array
    state_array = POMDPs.states(MyFactoredState, object_ranges, state_attr_ranges) 
    println("state array made ") 
    goal_states = find_goal_states(state_array)  
    println("goal states found")
    actions = temp_actions() 
    # Return a new GridWorldMDP with the precomputed state array
    return GameMDP(goal_states, 0.5 ,state_array, actions)
end

mdp = GameMDP()

state array made 
goal states found


GameMDP(MyFactoredState[MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 1,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))), MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 2,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))), MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 3,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))), MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 4,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))), MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 1,)), Blackball1 = ObjectState((pos = 2

In [247]:

POMDPs.actions(mdp::GameMDP) = temp_actions() 
POMDPs.states(mdp::GameMDP) = mdp.state_array

In [248]:
POMDPs.isterminal(mdp::GameMDP , s::MyFactoredState) = (s.attributes.Redball1Matched == 1 && s.attributes.Blackball1Matched == 1 )

# POMDPs.isterminal(mdp::GameMDP, s::MyFactoredState) =  ( s == MyFactoredState(
#     NamedTuple((Redball1Matched = -1, Blackball1Matched = -1)),
#     NamedTuple((Redball1 = ObjectState((pos = -1,)), Blackball1 = ObjectState((pos = -1,)), Redbox1 = ObjectState((pos = -1, )), Blackbox1 = ObjectState((pos = -1,))))
# )
# )

POMDPs.initialstate(mdp::GameMDP) = DeterministicState(
    MyFactoredState(
        NamedTuple((Redball1Matched = 0, Blackball1Matched = 0)),
        NamedTuple((Redball1 = ObjectState((pos = 1,)), Blackball1 = ObjectState((pos = 2,)), Redbox1 = ObjectState((pos = 3, )), Blackbox1 = ObjectState((pos = 4,))))
    )
    )


In [249]:

# Function that returns the index of a state in the state space array by matching
function POMDPs.stateindex(mdp::GameMDP, s::MyFactoredState)
    # Get the array of all states in the MDP
    state_array = mdp.state_array

    # Loop through the state array to find the matching state
    for (index, state) in enumerate(state_array)
        if state == s
            return index # Return the index when a match is found
        end
    end

    # If no matching state is found, return an error or a specific value
    error("State not found in the state space")
end


function POMDPs.actionindex(mdp::GameMDP, a::MatchAction)
    @assert in(a, mdp.actions ) "Invalid action"
    return findfirst(x -> x == a, mdp.actions  )
end


In [252]:
function POMDPs.transition(mdp::GameMDP, s::MyFactoredState, a::MatchAction)
    @assert in(a, mdp.actions ) "Invalid action"

    # define a temp state 
    temp_state =  MyFactoredState(
        NamedTuple((Redball1Matched = -1, Blackball1Matched = -1)),
        NamedTuple((Redball1 = ObjectState((pos = -1,)), Blackball1 = ObjectState((pos = -1,)), Redbox1 = ObjectState((pos = -1, )), Blackbox1 = ObjectState((pos = -1,))))
    )


    if (s in mdp.goal_states)
        return SparseCat([temp_state], [1.0])
    end 

    if (s == temp_state)
        return SparseCat([temp_state], [1.0])
    end


    if (a.attr1 == :Redball1 && a.attr2 == :Redbox1)
        # check preconditions
        if (s.attributes.Redball1Matched == 0 && s.objects.Redball1.attributes.pos != s.objects.Redbox1.attributes.pos)
            # create a new state
            new_state = deepcopy(s)
            new_attributes = merge(new_state.attributes, (Redball1Matched = 1,))

            new_redball1 = ObjectState(merge(
                new_state.objects.Redball1.attributes,
                (pos = s.objects.Redbox1.attributes.pos,)
            ))

            new_objects = (
                Redball1 = new_redball1,
                Blackball1 = new_state.objects.Blackball1,
                Redbox1 = new_state.objects.Redbox1,
                Blackbox1 = new_state.objects.Blackbox1
            )

            # Combine the updated named tuples to form the new state
            new_state = MyFactoredState(new_attributes, new_objects )

            return SparseCat([new_state], [1.0])

        else
            return SparseCat([temp_state], [1.0])
        end
    

    
    elseif (a.attr1 == :Blackball1 && a.attr2 == :Redbox1)
        # check preconditions
        if (s.attributes.Blackball1Matched == 0 && s.objects.Blackball1.attributes.pos != s.objects.Redbox1.attributes.pos)
            # create a new state
            new_state = deepcopy(s)
            
            new_blackball1 = ObjectState(merge(
                new_state.objects.Blackball1.attributes,
                (pos = s.objects.Redbox1.attributes.pos,)
            ))

            new_objects = (
                Redball1 = new_state.objects.Redball1,
                Blackball1 = new_blackball1,
                Redbox1 = new_state.objects.Redbox1,
                Blackbox1 = new_state.objects.Blackbox1
            )

            # Combine the updated named tuples to form the new state
            new_state = MyFactoredState(new_state.attributes, new_objects)

            return SparseCat([new_state], [1.0])
        

        else
            return SparseCat([temp_state], [1.0])
        end
    

    
    elseif (a.attr1 == :Redball1 && a.attr2 == :Blackbox1)
        # check preconditions
        if (s.attributes.Redball1Matched == 0 && s.objects.Redball1.attributes.pos != s.objects.Blackbox1.attributes.pos)
            # create a new state
            new_state = deepcopy(s)
            
            new_redball1 = ObjectState(merge(
                new_state.objects.Redball1.attributes,
                (pos = s.objects.Blackbox1.attributes.pos,)
            ))

            new_objects = (
                Redball1 = new_redball1,
                Blackball1 = new_state.objects.Blackball1,
                Redbox1 = new_state.objects.Redbox1,
                Blackbox1 = new_state.objects.Blackbox1
            )

            # Combine the updated named tuples to form the new state
            new_state = MyFactoredState(new_state.attributes, new_objects)

            return SparseCat([new_state], [1.0])
        
        else
            return SparseCat([temp_state], [1.0])
        end
    

   

    elseif (a.attr1 == :Blackball1 && a.attr2 == :Blackbox1)
        # check preconditions
        if (s.attributes.Blackball1Matched == 0 && s.objects.Blackball1.attributes.pos != s.objects.Blackbox1.attributes.pos)
            # create a new state
            new_state = deepcopy(s)
            
            new_attributes = merge(new_state.attributes, (Blackball1Matched = 1,))

            new_blackball1 = ObjectState(merge(
                new_state.objects.Blackball1.attributes,
                (pos = s.objects.Blackbox1.attributes.pos,)
            ))

            new_objects = (
                Redball1 = new_state.objects.Redball1,
                Blackball1 = new_blackball1,
                Redbox1 = new_state.objects.Redbox1,
                Blackbox1 = new_state.objects.Blackbox1
            )
           

            # Combine the updated named tuples to form the new state
            new_state = MyFactoredState(new_attributes, new_objects)
            
            return SparseCat([new_state], [1.0])
        

        else
            return SparseCat([temp_state], [1.0])
        end
    else 
        error("Invalid state action pair")
    end

end


In [251]:
print(mdp.goal_states)

MyFactoredState[MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 1,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))), MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 2,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))), MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 3,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))), MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 4,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))), MyFactoredState((Redball1Matched = 1, Blackball1Matched = 1), (Redball1 = ObjectState((pos = 1,)), Blackball1 = ObjectState((pos = 2,)), Red

Excessive output truncated after 524292 bytes.

), (Redball1 = ObjectState((pos = 3,)), Blackball1 = ObjectState((pos = 3,)), Redbox1 = ObjectState((pos = 3,)), Blackbox1 = ObjectState((

In [253]:

function POMDPs.reward(mdp::GameMDP, s::MyFactoredState, a::MatchAction, sp::MyFactoredState)
    if (sp in mdp.goal_states)
        return 1.0
    else
        return 0.0
    end
end

# Now we can define the reward function given the state and action
function POMDPs.reward(mdp::GameMDP, s::MyFactoredState, a::MatchAction)
    r = 0.0
    for (sp, p) in transition(mdp, s, a)
        r += p * reward(mdp, s, a, sp)
    end
    return r
end

In [254]:
function POMDPs.discount(mdp::GameMDP)
    return mdp.discount_factor
end

In [255]:
using POMDPLinter, POMDPModels, POMDPTools, DiscreteValueIteration
solver = ValueIterationSolver()
@show_requirements POMDPs.solve(solver, mdp)

true

In [260]:
solver = ValueIterationSolver(; max_iterations=100, verbose=true)
vi_policy = POMDPs.solve(solver, mdp)


ValueIterationPolicy:
 MyFactoredState((Redball1Matched = 0, Blackball1Matched = 0), (Redball1 = ObjectState((pos = 1,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))) -> MatchAction(:Redball1, :Redbox1)
 MyFactoredState((Redball1Matched = 0, Blackball1Matched = 0), (Redball1 = ObjectState((pos = 2,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))) -> MatchAction(:Redball1, :Redbox1)
 MyFactoredState((Redball1Matched = 0, Blackball1Matched = 0), (Redball1 = ObjectState((pos = 3,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))) -> MatchAction(:Redball1, :Redbox1)
 MyFactoredState((Redball1Matched = 0, Blackball1Matched = 0), (Redball1 = ObjectState((pos = 4,)), Blackball1 = ObjectState((pos = 1,)), Redbox1 = ObjectState((pos = 1,)), Blackbox1 = ObjectState((pos = 1,)))) -> MatchAction(:Redball1, 

In [267]:
s =  MyFactoredState(
    NamedTuple((Redball1Matched = 1, Blackball1Matched = 0)),
    NamedTuple((Redball1 = ObjectState((pos = 3,)), Blackball1 = ObjectState((pos = 2,)), Redbox1 = ObjectState((pos = 3, )), Blackbox1 = ObjectState((pos = 4,))))
)

@show action(vi_policy, s)

MatchAction(:Blackball1, :Blackbox1)

In [273]:
# print(s)
# POMDPs.transition(mdp, s,  MatchAction(:Blackball1, :Blackbox1))
# POMDPs.reward(mdp, s, MatchAction(:Blackball1, :Blackbox1))
POMDPs.reward(mdp, s, MatchAction(:Redball1, :Redbox1))

0.0